***This notebook trains our 4 adapters on the same dataset and compares their results.***

In [ ]:
import os
import random
import torch
from pathlib import Path
import numpy as np
from copy import deepcopy
from pprint import pprint as print

from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image
from dataclasses import dataclass
from typing import List, Literal, Optional, Tuple
from torch.utils.data.dataloader import DataLoader
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from effdet import create_model, create_loader
from effdet.data import resolve_input_config, resolve_fill_color
from effdet.bench import DetBenchPredict  # noqa F401
from effdet.data.transforms import ResizePad, ImageToNumpy, Compose
from timm.optim._optim_factory import create_optimizer_v2

from ml_carbucks.utils.preprocessing import create_clean_loader, create_transforms, preprocess_images
from ml_carbucks.utils.inference import plot_img_pred_subplots as psp
from ml_carbucks import DATA_DIR, RESULTS_DIR
from ml_carbucks.utils.postprocessing import (
    postprocess_prediction_nms,
    postprocess_evaluation_results,
)
from ml_carbucks.utils.result_saver import ResultSaver
from ml_carbucks.adapters.BaseDetectionAdapter import (
    ADAPTER_METRICS,
    BaseDetectionAdapter,
    ADAPTER_PREDICTION,
)
from ml_carbucks.patches.effdet import (
    CocoStatsEvaluator,
    ConcatDetectionDataset,
    create_dataset_custom,
)
from ml_carbucks.utils.conversions import convert_yolo_to_coco
from ml_carbucks.utils.logger import setup_logger
import json
import datetime as dt
from ml_carbucks.adapters.EfficientDetAdapter import EfficientDetAdapter
from ml_carbucks.adapters.FasterRcnnAdapter import FasterRcnnAdapter
from ml_carbucks.adapters.UltralyticsAdapter import (
    YoloUltralyticsAdapter,
    RtdetrUltralyticsAdapter,
)

logger = setup_logger(__name__)


DATASET_TUPLE_TYPE = tuple[str | Path, str | Path]

classes = ["scratch", "dent", "crack"]

cardd_dataset_train: DATASET_TUPLE_TYPE = (
    DATA_DIR / "car_dd" / "images" / "train",
    DATA_DIR / "car_dd" / "annotations" /"instances_train_curated.json",
)
cardd_dataset_val: DATASET_TUPLE_TYPE = (
    DATA_DIR / "car_dd" / "images" / "val",
    DATA_DIR / "car_dd" / "annotations" /"instances_val_curated.json",
)

carbucks_raw_dataset_train: DATASET_TUPLE_TYPE = (
    DATA_DIR / "carbucks" / "images" / "train",
    DATA_DIR / "carbucks" / "instances_train_curated.json",
)
carbucks_raw_dataset_val: DATASET_TUPLE_TYPE = (
    DATA_DIR / "carbucks" / "images" / "val",
    DATA_DIR / "carbucks" / "instances_val_curated.json",
)
carbucks_clean_dataset_train: DATASET_TUPLE_TYPE = (
    DATA_DIR / "carbucks_cleaned" / "images" / "train",
    DATA_DIR / "carbucks_cleaned" / "instances_train_curated.json",
)
carbucks_clean_dataset_val: DATASET_TUPLE_TYPE = (
    DATA_DIR / "carbucks_cleaned" / "images" / "val",
    DATA_DIR / "carbucks_cleaned" / "instances_val_curated.json",
)
carbucks_balanced_dataset_train : DATASET_TUPLE_TYPE = (
    DATA_DIR / "carbucks_balanced" / "images" / "train",
    DATA_DIR / "carbucks_balanced" / "instances_train_curated.json",
)
carbucks_balanced_dataset_val : DATASET_TUPLE_TYPE = (
    DATA_DIR / "carbucks_balanced" / "images" / "val",
    DATA_DIR / "carbucks_balanced" / "instances_val_curated.json",
)

mvp = {}

In [ ]:
def seed_everything(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)

EXPERIMENT_NAME = "car_dd_plus_carbucks_clean"
RUNTIME = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
RESULTS_ROOT = RESULTS_DIR / "debug" / "adapter_training" / f"{EXPERIMENT_NAME}_{RUNTIME}"
RESULTS_ROOT.mkdir(parents=True, exist_ok=True)

train_datasets = [cardd_dataset_train, carbucks_clean_dataset_train]
val_datasets = [cardd_dataset_val, carbucks_clean_dataset_val]

def run_adapter(
    adapter_cls,
    name: str,
    *,
    img_size: int = 512,
    epochs: int = 10,
    batch_size: int = 8,
    extra_params: dict | None = None,
):
    params = {"classes": classes, "img_size": img_size, "epochs": epochs, "batch_size": batch_size}
    if extra_params:
        params.update(extra_params)

    adapter = adapter_cls(**params).setup()
    adapter.fit(train_datasets)
    metrics = adapter.evaluate(val_datasets)

    save_dir = RESULTS_ROOT / name
    save_dir.mkdir(parents=True, exist_ok=True)
    adapter.save(save_dir, prefix=f"{name}_")
    (save_dir / "metrics.json").write_text(json.dumps(metrics, indent=2))
    print(f"{name}: {metrics}")
    return metrics

In [ ]:
efficientdet_metrics = run_adapter(
    EfficientDetAdapter,
    name="efficientdet_inbuild_optuna",
    extra_params={
        "img_size": 384,
        "batch_size": 16,
        "epochs": 30,
        "optimizer": "momentum",
        "lr": 4.175231383082291e-03,
        "weight_decay": 5.1636910035878785e-05,
        "confidence_threshold": 0.04424559135759042,
        "loader": "inbuild",
    },
)


In [ ]:
fasterrcnn_metrics = run_adapter(
    FasterRcnnAdapter,
    name="fasterrcnn_resnet50",
    img_size=512,
    epochs=10,
    batch_size=4,
    extra_params={
        "weights": "DEFAULT",
        "optimizer": "AdamW",
        "training_augmentations": True,
    },
)


In [ ]:
yolo_metrics = run_adapter(
    YoloUltralyticsAdapter,
    name="ultralytics_yolo11l",
    img_size=640,
    epochs=30,
    batch_size=16,
    extra_params={
        "weights": Path("/home/bachelor/ml-carbucks/yolo11l.pt"),
        "training_save": True,
        "project_dir": RESULTS_ROOT / "ultralytics",
        "name": "yolo11l_cardd_carbucks_clean",
        "training_augmentations": True,
    },
)


In [ ]:
rtdetr_metrics = run_adapter(
    RtdetrUltralyticsAdapter,
    name="ultralytics_rtdetr_l_optuna",
    extra_params={
        "img_size": 384,
        "batch_size": 32,
        "epochs": 14,
        "lr": 2.4406853367265458e-04,
        "momentum": 0.941920609133938,
        "weight_decay": 1.7052972123891876e-04,
        "optimizer": "NAdam",
        "weights": Path("/home/bachelor/ml-carbucks/rtdetr-l.pt"),
        "training_save": True,
        "project_dir": RESULTS_ROOT / "ultralytics",
        "name": "rtdetr_cardd_carbucks_clean_optuna",
    },
)


In [ ]:
all_metrics = {
    "efficientdet_inbuild": efficientdet_metrics,
    "fasterrcnn_resnet50": fasterrcnn_metrics,
    "ultralytics_yolo11l": yolo_metrics,
    "ultralytics_rtdetr_l": rtdetr_metrics,
}

for name, metrics in all_metrics.items():
    print(f"{name}: mAP50={metrics['map_50']:.4f}, mAP50-95={metrics['map_50_95']:.4f}")

import json
summary_path = RESULTS_ROOT / "metrics_summary.json"
summary_path.write_text(json.dumps(all_metrics, indent=2))
print(f"Saved summary to {summary_path}")
